In [ ]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
taskTime = 25 #task time (min)
rawNum = taskTime * 60 * 1000

binPath = path
with open(binPath, "rb") as f:
  data = f.read()
  rawData = np.frombuffer(data, dtype=np.float64)
  rawData = rawData.reshape(-1, 4)

if len(rawData) > rawNum:
  rawData = rawData[:rawNum]
  #[isosbestic_trigger, greenTri, flo_data]
  originalTime = [(i+1)/1000 for i in range (len(rawData))] # to second
  isoTri = rawData[:, 0]
  greenTri = rawData[:, 1]
  floData = rawData[:, 2] #include 405nm and 465nm

In [ ]:
isoTriLogi = []
greenTriLogi = []
triThreshold = 4
count = 0

triStartPoint = 2
triEndPoint = 19

#LED on:1
#Exclude 2msec at the start and end of excitation light on because of noise
for isoNum in range(len(isoTri)):
    if isoTri[isoNum] >= triThreshold:
      count += 1
      if triStartPoint < count < triEndPoint:
        isoTriLogi.append(1)
      else:
        isoTriLogi.append(0)
    else:
      count = 0
      isoTriLogi.append(0)

for greenNum in range(len(greenTri)):
    if greenTri[greenNum] >= triThreshold:
      count += 1
      if triStartPoint < count < triEndPoint:
        greenTriLogi.append(1)
      else:
        greenTriLogi.append(0)
    else:
      count = 0
      greenTriLogi.append(0)

isoTriLogi = np.array(isoTriLogi)
greenTriLogi = np.array(greenTriLogi)

#excitation start:1, excitation end:0
#Get start and end indexes
isoDiff = np.diff(isoTriLogi)
isoStartInd = [np.where(isoDiff == 1)[0] + 1][0]
isoEndInd = [np.where(isoDiff == -1)[0] + 1][0]

greenDiff = np.diff(greenTriLogi)
greenStartInd = [np.where(greenDiff == 1)[0] + 1][0]
greenEndInd = [np.where(greenDiff == -1)[0] + 1][0]

In [ ]:
#Remove bugs
if greenStartInd[0] - greenEndInd[0] > 0:
  greenEndInd = greenEndInd[1:]
elif isoStartInd[0] - isoEndInd[0] > 0:
  isoEndInd = isoEndInd[1:]

if greenStartInd[-1] - greenEndInd[-1] > 0:
  greenStartInd = greenStartInd[:-1]
elif isoStartInd[-1] - isoEndInd[-1] > 0:
  isoStartInd = isoStartInd[:-1]

if len(isoStartInd) > len(greenStartInd):
  isoStartInd = isoStartInd[1:]
  isoEndInd = isoEndInd[1:]
elif len(isoStartInd) < len(greenStartInd):
  greenStartInd = greenStartInd[1:]
  greenEndInd = greenEndInd[1:]

In [ ]:
isoMean = []
greenMean = []
greenMeanTime = []

#Calculate the average of the fluorescence
try:
  for i in range(len(isoStartInd)):
    isoMean.append(np.mean(floData[isoStartInd[i]:isoEndInd[i]]))
except IndexError:
  pass

try:
  for i in range(len(greenStartInd)):
    greenMean.append(np.mean(floData[greenStartInd[i]:greenEndInd[i]]))
    greenMeanTime.append(np.mean(originalTime[greenStartInd[i]:greenEndInd[i]]))
except IndexError:
  pass

if len(isoMean) < len(greenMean):
  greenMean = greenMean[:len(isoMean)]
  greenMeanTime = greenMeanTime[:len(isoMean)]

In [ ]:
#Calculate moving average
convNum = np.ones(20)/ 20
isoMovingAve = np.convolve(isoMean, convNum, mode="same")
greenMovingAve = np.convolve(greenMean, convNum, mode="same")

#Make time list
timeList = [i*0.05 for i in range(1, len(isoMovingAve)+1, 1)]

In [ ]:
floMovingAve = []
floMovingAve.append(timeList)
floMovingAve.append(isoMovingAve)
floMovingAve.append(greenMovingAve)

df = pd.DataFrame(floMovingAve, index=["time", "iso", "green"]).T